In [11]:
import fitz # PyMuPDF 
import os 
import requests 
from transformers import LlamaForQuestionAnswering, LlamaTokenizer, AutoModel

In [12]:
pdf_path = '../../data/external/Illustrated Encyclopedia of Birds Part 1.pdf'

In [33]:
# Text-Extraktion 
def extract_text_from_pdf(pdf_path, num_pages=None): 
    text = '' 
    with fitz.open(pdf_path) as pdf:
        doc = pdf
        if num_pages is None:
            num_pages = len(doc)
        for i in range(min(num_pages, len(doc))): 
            page = doc.load_page(i) 
            text += page.get_text()
    return text

In [36]:
print(extract_text_from_pdf(pdf_path, num_pages=27))

I L L U S T R A T E D  E N C Y C L O P E D I A  O F
I L L U S T R A T E D  E N C Y C L O P E D I A  O F
I L L U S T R A T E D  E N C Y C L O P E D I A  O F
INTRODUCTION
LONDON, NEW YORK, MELBOURNE, 
MUNICH AND DELHI
CONTENTS
SCHERMULY DESIGN COMPANY
ABOUT THIS BOOK  
    6
FOREWORD by dr marco lambertini, 
birdlife international 
    9
BIRD ANATOMY 
24
SENSES  
26
FEATHERS 
28
WINGS  
30
FLIGHT  
32
GLIDING AND SOARING 
34
LEGS AND FEET 
36
BILLS 
 
38
FEEDING ON ANIMALS 
40
FEEDING ON PLANTS 
42
COMMUNICATION 
44
DEFENCE 
46
BREEDING 
48
COURTSHIP 
50
NESTS AND EGGS 
52
PARENTAL CARE 
54
LIVING TOGETHER 
56
MIGRATION 
58
MIGRATION ROUTES 
60
BIRDS UNDER THREAT 
62
CONSERVATION 
64
EXTINCT BIRDS 
66
BIRD GEOGRAPHY 
70
BIRD HABITATS 
72
GRASSLANDS 
74
DESERTS  
76
TROPICAL FOREST 
78
TEMPERATE FOREST 
80
CONIFEROUS FOREST 
82
HEATH AND SCRUBLAND 
84
MOUNTAINS 
86
POLAR REGIONS 
88
WETLANDS 
90
COASTS  
92
OCEANS AND SEAS 
94
FARMLAND AND CITIES 
96
HABITATS
SENIOR EDITOR
Peter Frances
M

In [14]:
# Segmentierung (z.B. nach Absätzen) 
def segment_text(text): 
    return text.split('\n\n')

In [15]:
# Indexierung (einfacher Ansatz mit Dictionary) 
def create_index(segments): 
    index = {} 
    for i, segment in enumerate(segments): 
        index[i] = segment 
    return index

In [16]:
# Funktion zur Abfragebeantwortung mit lokalem LLaMA-Modell 
def answer_query_with_llama(query, index): 
    # LLaMA-Modell und Tokenizer laden (anpassen auf Ihr Modell und Checkpoint) 
    model = AutoModel.from_pretrained('llama3.2:3b-instruct-q8_0') 
    tokenizer = LlamaTokenizer.from_pretrained('llama3.2:3b-instruct-q8_0') 
    
    # Relevante Segmente finden (einfacher Ansatz) 
    relevant_segments = [segment for segment in index.values() if query.lower() in segment.lower()] 
    
    # Antworten generieren (nur ein Beispiel, möglicherweise Anpassungen erforderlich) 
    answers = [] 
    for segment in relevant_segments: 
        inputs = tokenizer(query, segment, return_tensors='pt') 
        outputs = model(**inputs) 
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True) 
        answers.append(answer) 
    return answers 

In [17]:
# Anwendung des Codes 
pdf_text = extract_text_from_pdf(pdf_path) 
segments = segment_text(pdf_text) 
index = create_index(segments) 
query = 'Tell me something about the anatomy of birds.' 
answers = answer_query_with_llama(query, index) 
for i, answer in enumerate(answers): 
    print(f'Antwort {i+1}: {answer}')

OSError: Incorrect path_or_model_id: 'llama3.2:3b-instruct-q8_0'. Please provide either the path to a local folder or the repo_id of a model on the Hub.